In [73]:
# Librerias
import random
import numpy as np
from matplotlib import use
import matplotlib.pyplot as plt
import sympy as sp
#from sympy import symbols, Eq, Point3D, Plane, solve, Line3D
import pandas as pd
from shapely.geometry import Polygon, Point

use('Qt5Agg')

In [74]:
inter = None
color_t = None
caras = None
vertex = None
normal = None
M = None
A = None
mp = None
eigenvalues = None
eigenvectors = None
axis = None
traces = None
tri = None
color_m = None
size = None
max_value = 0
x_axis = None
y_axis = None
squares = None

In [75]:
def new_plot():
    global axis
    fig = plt.figure()
    axis = fig.add_subplot(111, projection='3d')

def build_box():
        lines = [
            [0, 1], [0, 2], [0, 4],
            [1, 3], [1, 5], [2, 3],
            [2, 6], [3, 7], [4, 5],
            [4, 6], [5, 7], [6, 7]
        ]
        for line in lines:
            axis.plot(
                [vertex[line[0], 0], vertex[line[1], 0]],
                [vertex[line[0], 1], vertex[line[1], 1]],
                [vertex[line[0], 2], vertex[line[1], 2]],
                color='black'
            )

def print_plot():
    axis.set_xlabel('X')
    axis.set_ylabel('Y')
    axis.set_zlabel('Z')

    axis.legend()
    plt.xlim(x_axis - max_value, x_axis + max_value)
    plt.ylim(y_axis - max_value, y_axis + max_value)
    plt.show()

In [76]:
# Carga los datos de la fractura desde un archivo de texto.
dataset = []
with open("FRAC0006_nrIter27.txt", 'r') as file:
    for line in file:
        values = [float(value) for value in line.split()]
        dataset.append(values)
M = np.array(dataset)

In [77]:
def limits():
    global x_axis, y_axis, max_value    
    min_limits = np.min(M, axis=0)
    max_limits = np.max(M, axis=0)

    for i in range(3):
        max_value = max(max_value, abs(max_limits[i] - min_limits[i]))
    max_value/=2
    x_axis = np.min(M[:, 0]) + max_value/2
    y_axis = np.min(M[:, 1]) + max_value/2
    print(x_axis, y_axis, max_value)
    print(x_axis - max_value, x_axis + max_value)
    print(y_axis - max_value, y_axis + max_value)
limits()

-14.847661413249998 -8.41543555325 4.7660019535
-19.613663366749996 -10.081659459749998
-13.18143750675 -3.649433599749999


In [78]:
"""
    Calcula la matriz A (matriz de covarianza) de una matriz M y
    los autovalores y autovectores
"""
def load_matrix():
    global A, mp, M
    print(len(M))
    M = M[400:440]

    A = np.dot(M.T, M)

    mp = np.mean(M, axis=0)

    eigenvalues, eigenvectors = np.linalg.eig(A)

load_matrix()

540


In [79]:
def get_box():
    """
        Calcula los vértices de la caja que encierra la fractura (matriz m)
    """
    global vertex
    points = M - mp
    _eigenvalues, _eigenvectors = np.linalg.eig(np.cov(points, rowvar=False))
    points = np.dot(points, _eigenvectors)

    # Calcula los límites de la caja en el nuevo sistema de coordenadas
    min_limits = np.min(points, axis=0)
    max_limits = np.max(points, axis=0)

    # Crear puntos de los límites de la caja en el sistema transformado
    vertex = np.array(np.meshgrid([min_limits[0], max_limits[0]], [min_limits[1], max_limits[1]],
                                        [min_limits[2], max_limits[2]])).T.reshape(-1, 3)

    # Transformar los vértices de la caja de vuelta al sistema original
    vertex = np.dot(vertex, _eigenvectors.T) + mp

get_box()

In [80]:
def color_points():
    global color_m, size
    color_m = []
    size = int(len(M) / 20)
    for i in range(size):
        r = [random.random(), random.random(), random.random()]
        color_m.append(r)

    color_m = np.array(color_m)

color_points()

In [81]:
"""
    Representación de los datos en 3D utilizando Matplotlib
"""
def print_points():
    for i in range(size):
        m = M[i * 20:i * 20 + 20]
        axis.scatter(m[:, 0], m[:, 1], m[:, 2], color=color_m[i], s=2)
    

new_plot()
build_box()
print_points()
print_plot()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [102]:
def triangularization():
    """
        Realizar la triangulación de los puntos, teniento en cuenta que cada 20 puntos se apueden realizar
        triangulaiones con los siguientes 2o puntos.

        tri alamacena la dirección de los puntos que forman un triangulo
    """
    global color_t, tri
    color_t = []
    tri = []
    for i in range(0, size - 1):
        for j in range(20):
            if j == 19:
                tri.append([j + (20 * i), j + (20 * (i + 1)), 0 + (20 * (i + 1))])
                tri.append([j + (20 * i), 0 + (20 * (i + 1)), 0 + (20 * i)])
            else:
                tri.append([j + (20 * i), j + (20 * (i + 1)), (j + 1) + (20 * (i + 1))])
                tri.append([j + (20 * i), (j + 1) + (20 * (i + 1)), (j + 1) + (20 * i)])

            color = (color_m[i] + 2 * color_m[i + 1]) / 3
            color_t.append(color)
            color = (2 * color_m[i] + color_m[i + 1]) / 3
            color_t.append(color)
    color_t = np.array(color_t)
    tri = np.array(tri)

triangularization()

In [83]:
"""
    graficar triangularizacion
"""
def print_triangles():
    for s in range(len(tri)):
        triangle = np.array([M[tri[s][0]], M[tri[s][1]], M[tri[s][2]], M[tri[s][0]]])
        axis.plot(triangle[:, 0], triangle[:, 1], triangle[:, 2], color=color_t[s])

new_plot()
build_box()
print_triangles()
print_plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [103]:
def norm():
    """
    sacar la norma de cada triangulo
    """
    global normal
    normal = []
    aux = 0
    for triangle in tri:
        p1, p2, p3 = M[triangle]
        v1 = p2 - p1
        v2 = p3 - p1
        n = np.cross(v1, v2)
        normal.append(n / np.linalg.norm(n))

norm()

C:\Users\julia\AppData\Local\Temp\ipykernel_16336\968828340.py:13: RuntimeWarning: invalid value encountered in divide
  normal.append(n / np.linalg.norm(n))


In [108]:
def print_norm():
        """
        graficar la norma de los triangulos
        """
        for i in range(len(normal)):
            mp = np.mean(M[tri[i]], axis=0)
            v = 0.3 * normal[i]
            axis.quiver(mp[0], mp[1], mp[2], v[0], v[1], v[2], color='b')

new_plot()
build_box()
print_norm()
print_plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [104]:
def dropnan():
    """
        Elimina los valores nulos de la matriz de los triangulos
    """
    global tri, normal, color_t    
    df_t = pd.concat([pd.DataFrame(tri), pd.DataFrame(normal), pd.DataFrame(color_t)], axis = 1)
    df_t.columns = ["triangles_x", "triangles_y", "triangles_z",
                    "norm_x", "norm_y", "norm_z",
                    "r", "g", "b"]

    if df_t.isnull().any().any():
        df_t = df_t.dropna()
        tri = df_t[["triangles_x", "triangles_y", "triangles_z"]].to_numpy()
        normal = df_t[["norm_x", "norm_y", "norm_z"]].to_numpy()
        color_t = df_t[["r", "g", "b"]].to_numpy()

dropnan()


In [118]:
caras = []
points = [[0, 1, 2, 3], [0, 1, 4, 5], [0, 4, 2, 6], [7, 5, 3, 1], [7, 6, 3, 2], [7, 6, 5, 4]]
def plane_from_points():
    """
        Calcula los planos a partir de puntos dados y los guarda en la lista.
    """
    # points = [[0, 1, 4, 5]]
    # points = [[0, 1, 2, 3], [0, 1, 4, 5], [0, 4, 2, 6], [7, 5, 3, 1], [7, 6, 3, 2], [7, 6, 5, 4]]
    # points = [[0, 1, 4], [7, 6, 5], [7, 6, 3], [0, 4, 2], [7, 5, 3], [0, 1, 2]]
    for p in points:
        p1, p2, p3 = map(sp.Point3D, (vertex[p[0]], vertex[p[1]], vertex[p[2]]))
        caras.append(sp.Plane(p1, p2, p3))

def line_from_point_and_vector(point, vector):
    """
        Crea una línea a partir de un punto y un vector.

        Args:
        - point: Punto inicial de la línea.
        - vector: Vector director de la línea.

        Returns:
        - Línea creada.
    """
    point = sp.Point3D(point)
    direction_point = point + sp.Point3D(*vector)
    line = sp.Line3D(point, direction_point)
    return line

def direction_points(pi, vd, d):
    """
        Calcula un punto final dado un punto inicial, un vector director y una distancia.

        Returns:
        - Punto final calculado.
        """
    norma_vector = np.linalg.norm(vd)
    direc = vd / norma_vector
    return pi + d * direc

def intersection_plane_line(line, mp, v):
        """
            Args:
            - line: Línea con la que se calcula la intersección.

            Returns:
            - Lista de puntos de intersección, si hay alguno, o None en caso contrario.
        """
        intersections = []
        for plane in range(len(caras)):
            i = caras[plane].intersection(line)
            if len(i) == 1 and direction_point(mp, i, v) and point_in_square(i, points[plane]):  # Solo hay un punto de intersección
                print(True)
                # intersection_point = intersection[0]
                return i
                # return inter
            print(False)
        if len(intersections) > 0:
            return intersections
        else:
            return None
        
def point_in_square(p, s):
    """
        Comprueba si un punto está dentro de un cuadrado.
    """
    c = np.array([vertex[s[0]], vertex[s[1]], vertex[s[2]], vertex[s[3]]])
    d = np.array([abs(c[0][i]-c[1][i]) for i in range(3)])
    min_p = np.argmin(d)
    c[:, min_p] = np.floor(c[:,min_p])
    point = [np.array([p[0].x]), np.array([p[0].y]), np.array([p[0].z])]
    point[min_p] = np.floor(point[min_p])
    point = Point(point)
    polygon = Polygon(c)
    x = polygon.contains(point) or polygon.touches(point)
    return x

def distance(inter, pi, pr):
    """
        Calcula la distancia entre dos puntos.
    """
    i = np.array([inter[0].x, inter[0].y, inter[0].z])
    return (i[0] - pr[0])**2 + (i[1] - pr[1])**2 + (i[2] - pr[2])**2 < (i[0] - pi[0])**2 + (i[1] - pi[1])**2 + (i[2] - pi[2])**2
import numpy as np

def direction_point(pi, inter, v):
    # Convertir los puntos y el vector en arrays de numpy
    pi = np.array(pi)
    inter = np.array(inter)
    v = np.array(v)
    
    # Calcular el vector desde el punto inicial al punto de intersección
    v_inter = inter - pi
    
    # Calcular el producto punto
    pp = np.dot(v_inter, v)
    
    # Verificar si el producto punto es positivo
    print(pp)
    if pp < 0.22 and  pp > -0.22:
        print(pp)
        return True
    else:
        return False 

In [119]:
def intersection():
    """
        Calcula las intersecciones entre las normales de los triángulos y la caja, y guarda los puntos en la lista
        inter.
    """
    global inter
    inter = []
    plane_from_points()
    aux = 0
    for i in range(len(normal)):
        mp = np.mean(M[tri[i]], axis=0)
        v = normal[i]
        line = line_from_point_and_vector(mp, v)
        # op = direction_points(mp, v, 1)
        p = intersection_plane_line(line, mp, v)
        print(aux)
        aux += 1
        if p is None:
            print("Vacio") 
            continue
        p = [p[0].x, p[0].y, p[0].z]
        inter.append(p)
    inter = np.array(inter)

intersection()

[-230.011923981110]
False
[-0.139207845985311]
[-0.139207845985311]
True
0
[-29.2541117926085]
False
[-0.143008349524739]
[-0.143008349524739]
True
1
[-9.04175018373546]
False
[-0.145387921966260]
[-0.145387921966260]
True
2
[-3.18800697984598]
False
[-0.150893433902821]
[-0.150893433902821]
True
3
[-3.04518928352323]
False
[-0.145203334471419]
[-0.145203334471419]
True
4
[-2.14146889548551]
False
[-0.140373806053998]
[-0.140373806053998]
True
5
[-5.68803202927355]
False
[-0.130602254576194]
[-0.130602254576194]
True
6
[-10.8864435958625]
False
[-0.121642379910465]
[-0.121642379910465]
True
7
[-18.0618060354246]
False
[-0.113396464142764]
[-0.113396464142764]
True
8
[-36.2556525636042]
False
[-0.105096974129824]
[-0.105096974129824]
True
9
[-66.2928204674677]
False
[-0.0987970739535915]
[-0.0987970739535915]
True
10
[-194.846729597258]
False
[-0.0951999123141990]
[-0.0951999123141990]
True
11
[781.760645203325]
False
[-0.0945711431452494]
[-0.0945711431452494]
True
12
[222.175299402606

In [115]:
print(len(inter))

29


In [112]:
def print_intersection():
    """
        Añade los puntos de intersección al gráfico.
    """
    global color_t
    # color_t = color_t[:len(inter)]
    axis.scatter(inter[:, 0], inter[:, 1], inter[:, 2], color=color_t, s=2)
new_plot()
build_box()
print_intersection()
print_plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [91]:
def square():
    """
        Calcula los cuadrados a partir de los puntos de intersección y los guarda en la lista squares.
    """
    global squares
    sq = []
    for i in range(0, len(inter), 2):
        if (i + 2) % 40 == 0:
            square = [i, i+1, i+2-40, i+3-40]
        else:
            square = [i, i+1, i+2, i+3]
        sq.append(square)
    sq = np.array(sq)
    sq2 = []
    for i in range(41, len(inter), 2): 
        if (i+1) % 40 == 0:
            sq2.append([i, i-41, i-38, i-79])
        else:
            sq2.append([i, i-41, i+2, i-39])
    sq2= np.array(sq2)
    squares = sq[:20]
    for i in range(1, int(len(sq)/20)):
        squares = np.concatenate((squares, sq[i*20:i*20+20], sq2[:i*20]))

square()
print(len(squares))

8


In [92]:
color_s = []
def color_square():
    """
        Calcula el color de los cuadrados y los guarda en la lista color_s.
    """
    global color_s
    for i in squares:
        color = (color_t[i[0]] + color_t[i[1]] + color_t[i[2]] + color_t[i[3]]) / 4
        color_s.append(color)
    color_s = np.array(color_s)
color_square()
print(len(color_s))

IndexError: index 15 is out of bounds for axis 0 with size 15

In [ ]:
def print_squares():
    """
        Grafica los cuadrados.
    """
    for square in squares:
        s = [inter[square[0]], inter[square[1]], inter[square[3]], inter[square[2]], inter[square[0]]]
        axis.plot([v[0] for v in s], [v[1] for v in s], [v[2] for v in s], color="b")

In [ ]:
mp_s = []
def square_prom():
    """
        Calcula el punto promedio de cada cuadrado y los guarda en la lista mp_s.
    """
    global mp_s
    for i in squares:
        mp = np.mean(inter[i], axis=0)
        mp_s.append(mp)
    mp_s = np.array(mp_s)

square_prom()
print(len(mp_s))

5060
-261318228631940180057404125577783984388405745705711756049982243356391226608623708337606156405688236132418537014759941049609712259855869418787468852836994531896974865124726581008834946289343542133167/15253514937820943394935188817060116001353689774398035620732233860798814255971868708992297213830391763388976749867336747017322317095464198118844243647262562218698787890041146924861334000000000000000


In [ ]:
def print_p_square():
    """
        Grafica los puntos promedio de los cuadrados.
    """
    axis.scatter(mp_s[:, 0], mp_s[:, 1], mp_s[:, 2],s=1, color=color_s)

new_plot()
build_box()
print_p_square()
print_squares()
print_plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
